In [29]:
import librosa
import sounddevice as sd
import numpy as np
import os

# Wczytaj przykładowy plik audio z Librosy
data, sr = librosa.load(librosa.ex('nutcracker'))

In [30]:
def add_echo_to_audio(data, alpha, delta):
    """Dodaje echo do sygnału audio.

    Kopiuje sygnał i dodaje echo do każdego kanału (mono lub stereo) według formuły: x[n] + alpha * x[n - delta].
    Obsługuje sygnały mono i stereo. Nie normalizuje sygnału.

    Args:
        data (numpy.ndarray): Sygnał audio, tablica 1D (mono) lub 2D (stereo, shape: [próbki, kanały]).
        alpha (float): Siła echa (np. 0.4 dla umiarkowanego echa).
        delta (int): Opóźnienie echa w próbkach (np. 100 dla ~2.3 ms przy 44.1 kHz).

    Returns:
        numpy.ndarray: Sygnał z dodanym echem, ten sam kształt co wejściowy.

    Raises:
        ValueError: Gdy data jest pusta lub delta jest ujemne/zero.
        TypeError: Gdy data nie jest tablicą numpy.ndarray.

    Example:
        >>> import numpy as np
        >>> signal = np.zeros(1000)  # Mono, 1000 próbek
        >>> signal[0] = 1.0
        >>> echoed = add_echo_to_audio(signal, alpha=0.4, delta=100)
        >>> print(echoed[100])  # Powinno być 0.4
    """
    
    # Sygnał wejściowy
    output = np.copy(data)

    if len(data.shape) > 1:
        # Dodaj echo do każdego kanału: x[n] + alpha * x[n - delta]
        for channel in range(data.shape[1]):  # Iteruj po kanałach (0: lewy, 1: prawy)
            for i in range(delta, len(data)):
                output[i, channel] += alpha * data[i - delta, channel]
    else:
        # Dodaj echo do kanału
        for i in range(delta, len(data)):
                output[i] += alpha * data[i - delta]
            
    return output

In [31]:
def add_echo_to_file(input_file, output_file, alpha, delta):
    """Dodaje echo do pliku audio.

    Wczytuje plik audio, dodaje echo do każdego kanału sygnału i zapisuje wynik w rozszerzeniu pliku wyjściowego.
    Echo jest dodawane jako: x[n] + alpha * x[n - delta]. Sygnał jest normalizowany,
    aby uniknąć przesterowania.

    Args:
        input_file (str): Ścieżka do wejściowego pliku audio (mono lub stereo).
        output_file (str): Ścieżka do pliku wyjściowego WAV.
        alpha (float): Siła echa (np. 0.4 dla umiarkowanego echa).
        delta (int): Opóźnienie echa w próbkach (np. 100 dla ~2.3 ms przy 44.1 kHz).

    Returns:
        None: Funkcja zapisuje plik WAV z echem.

    Raises:
        FileNotFoundError: Gdy plik wejściowy nie istnieje.
        RuntimeError: Gdy zapis pliku wyjściowego nie powiedzie się.

    Example:
        >>> add_echo("input.wav", "output.wav", alpha=0.4, delta=100)
        # Dodaje echo o sile 0.4 z opóźnieniem 100 próbek do input.wav
        # Zapisuje wynik w output.wav
    """
    
    # Wczytaj plik audio
    data, sr = librosa.load(input_file)

    # Dodaj echo do sygnału
    output = add_echo_to_audio(data, alpha, delta)

    # Normalizuj sygnał, aby uniknąć przesterowania
    output = np.clip(output, -32768, 32767)  # Dla 16-bitowego WAV

    # Zapisz plik wyjściowy
    extension = os.path.splitext(output_file)[-1].lower().lstrip(".")
    output.export(output_file, format=extension)

In [32]:
alpha = 0.4
delta = 75
output = add_echo_to_audio(data, alpha, delta)

In [ ]:
sd.play(output, sr)
sd.wait()